In [34]:
import numpy as np
import pandas as pd
from datetime import timedelta
import Levenshtein
pd.options.mode.chained_assignment = None

## Problem statement
For every state, find the date on which the entire population will get at least one does of vaccination. Assume the same rate of vaccination as in the week ending on 14th Aug, 2021. (Do not treat children separately, and assume the same rate of vaccination.)
Output them in the following manner: stateid, populationleft, rateofvaccination, date.

In [35]:
vacc_data1 = pd.read_csv('../data/cowin_vaccine_data_districtwise.csv', low_memory=False)
vacc_data1

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9
0,NaN,NaN,NaN,NaN,NaN,NaN,Total Individuals Registered,Sessions,Sites,First Dose Administered,...,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Covaxin (Doses Administered),CoviShield (Doses Administered)
1,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
vacc_data1.iloc[0]

S No                                            NaN
State_Code                                      NaN
State                                           NaN
District_Key                                    NaN
Cowin Key                                       NaN
District                                        NaN
16/01/2021             Total Individuals Registered
16/01/2021.1                               Sessions
16/01/2021.2                                 Sites 
16/01/2021.3                First Dose Administered
16/01/2021.4               Second Dose Administered
16/01/2021.5           Male(Individuals Vaccinated)
16/01/2021.6         Female(Individuals Vaccinated)
16/01/2021.7    Transgender(Individuals Vaccinated)
16/01/2021.8           Covaxin (Doses Administered)
16/01/2021.9        CoviShield (Doses Administered)
17/01/2021             Total Individuals Registered
17/01/2021.1                               Sessions
17/01/2021.2                                 Sites 
17/01/2021.3

In [37]:
vacc_data = vacc_data1.iloc[1:]
vacc_data.fillna(0, inplace=True)
col = list(vacc_data.columns[6:])
vacc_data[col] = vacc_data[col].astype(int)
vacc_data

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9
1,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,0,0,0,0,0,0,0,0,0,0
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,0,0,0,0,0,0,0,0,0,0
5,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,0,0,0,0,0,0,0,0,0,0
6,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,0,0,0,0,0,0,0,0,0,0
7,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,0,0,0,0,0,0,0,0,0,0
8,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,0,0,0,0,0,0,0,0,0,0
9,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,0,0,0,0,0,0,0,0,0,0
10,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,0,0,0,0,0,0,0,0,0,0


In [38]:
distr_data = pd.read_csv('../output/edge-graph.csv', index_col='District_Key')
distr_data.drop(columns=['Unnamed: 0'], inplace=True)
districts_key = list(distr_data.index)
districts_key

['AP_Anantapur',
 'AP_Chittoor',
 'AP_East Godavari',
 'AP_Guntur',
 'AP_Krishna',
 'AP_Kurnool',
 'AP_Prakasam',
 'AP_S.P.S. Nellore',
 'AP_Srikakulam',
 'AP_Visakhapatnam',
 'AP_Vizianagaram',
 'AP_West Godavari',
 'AP_Y.S.R. Kadapa',
 'AR_Anjaw',
 'AR_Changlang',
 'AR_East Kameng',
 'AR_East Siang',
 'AR_Kamle',
 'AR_Kra Daadi',
 'AR_Kurung Kumey',
 'AR_Lepa Rada',
 'AR_Lohit',
 'AR_Longding',
 'AR_Lower Dibang Valley',
 'AR_Lower Siang',
 'AR_Lower Subansiri',
 'AR_Namsai',
 'AR_Pakke Kessang',
 'AR_Papum Pare',
 'AR_Shi Yomi',
 'AR_Siang',
 'AR_Tawang',
 'AR_Tirap',
 'AR_Upper Dibang Valley',
 'AR_Upper Siang',
 'AR_Upper Subansiri',
 'AR_West Kameng',
 'AR_West Siang',
 'AS_Baksa',
 'AS_Barpeta',
 'AS_Biswanath',
 'AS_Bongaigaon',
 'AS_Cachar',
 'AS_Charaideo',
 'AS_Chirang',
 'AS_Darrang',
 'AS_Dhemaji',
 'AS_Dhubri',
 'AS_Dibrugarh',
 'AS_Dima Hasao',
 'AS_Goalpara',
 'AS_Golaghat',
 'AS_Hailakandi',
 'AS_Hojai',
 'AS_Jorhat',
 'AS_Kamrup',
 'AS_Kamrup Metropolitan',
 'AS_Karbi

In [39]:
map_dkey_sname, map_dkey_dname = {}, {}
for dt in districts_key:
    map_dkey_sname[dt] = vacc_data[vacc_data['District_Key'] == dt]['State'].iloc[0]
    map_dkey_dname[dt] = vacc_data[vacc_data['District_Key'] == dt]['District'].iloc[0]
map_dkey_dname

{'AP_Anantapur': 'Anantapur',
 'AP_Chittoor': 'Chittoor',
 'AP_East Godavari': 'East Godavari',
 'AP_Guntur': 'Guntur',
 'AP_Krishna': 'Krishna',
 'AP_Kurnool': 'Kurnool',
 'AP_Prakasam': 'Prakasam',
 'AP_S.P.S. Nellore': 'S.P.S. Nellore',
 'AP_Srikakulam': 'Srikakulam',
 'AP_Visakhapatnam': 'Visakhapatnam',
 'AP_Vizianagaram': 'Vizianagaram',
 'AP_West Godavari': 'West Godavari',
 'AP_Y.S.R. Kadapa': 'Y.S.R. Kadapa',
 'AR_Anjaw': 'Anjaw',
 'AR_Changlang': 'Changlang',
 'AR_East Kameng': 'East Kameng',
 'AR_East Siang': 'East Siang',
 'AR_Kamle': 'Kamle',
 'AR_Kra Daadi': 'Kra Daadi',
 'AR_Kurung Kumey': 'Kurung Kumey',
 'AR_Lepa Rada': 'Lepa Rada',
 'AR_Lohit': 'Lohit',
 'AR_Longding': 'Longding',
 'AR_Lower Dibang Valley': 'Lower Dibang Valley',
 'AR_Lower Siang': 'Lower Siang',
 'AR_Lower Subansiri': 'Lower Subansiri',
 'AR_Namsai': 'Namsai',
 'AR_Pakke Kessang': 'Pakke Kessang',
 'AR_Papum Pare': 'Papum Pare',
 'AR_Shi Yomi': 'Shi Yomi',
 'AR_Siang': 'Siang',
 'AR_Tawang': 'Tawang'

In [40]:
pop_data = pd.read_excel('../data/DDW_PCA0000_2011_Indiastatedist.xlsx')
pop_data = pop_data[['Name', 'Level', 'TRU', 'TOT_P']]
pop_data = pop_data[pop_data['TRU'] == 'Total']
pop_data.drop(columns=['TRU'], inplace=True)
pop_data

,Name,Level,TOT_P
0,India,India,1210854977
3,JAMMU & KASHMIR,STATE,12541302
6,Kupwara,DISTRICT,870354
9,Badgam,DISTRICT,753745
12,Leh(Ladakh),DISTRICT,133487
15,Kargil,DISTRICT,140802
18,Punch,DISTRICT,476835
21,Rajouri,DISTRICT,642415
24,Kathua,DISTRICT,616435
27,Baramula,DISTRICT,1008039


In [41]:
overall_pop_data = pop_data[pop_data['Level'] == 'India']
overall_pop_data

,Name,Level,TOT_P
0,India,India,1210854977


In [42]:
vacc_data[vacc_data['District_Key'] == 'AP_Anantapur'].iloc[:, :100]

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,24/01/2021.4,24/01/2021.5,24/01/2021.6,24/01/2021.7,24/01/2021.8,24/01/2021.9,25/01/2021,25/01/2021.1,25/01/2021.2,25/01/2021.3
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,0,535,3861,0,0,4396,25439,529,111,4923


In [43]:
# This function will find the most similar district word from the given district word in the dataset.
def get_most_similar_word(df, w, name):
    '''
    inputs:
        df : input data containing all the state and districts code i.e DataFrame format
        w : district name
    output:
        output1 : most similar district or state
        output2 : similarity score between two given word input and most similar word
        
    '''
    temp_df = df.copy()
    temp_df = temp_df[1:]
    temp_df['Levenshtein_dist'] = temp_df[name].apply(lambda x: Levenshtein.distance(w, x))
    temp_df = temp_df.sort_values(by=['Levenshtein_dist'])
#     print(temp_df['District'].iloc[0])
    #print("with value: ", temp_df['Levenshtein_dist'].iloc[0])
    return temp_df[name].iloc[0], temp_df['Levenshtein_dist'].iloc[0]

In [44]:
sdf = pop_data.copy()
sdf = sdf[sdf['Level'] == 'STATE']
is_available = {'DAMAN & DIU':'Dadra and Nagar Haveli and Daman and Diu', 
                'DADRA & NAGAR HAVELI': 'Dadra and Nagar Haveli and Daman and Diu'}
for st in sdf['Name'].unique():
    if st in is_available.keys():
        sdf.loc[sdf['Name'] == st, 'State_Code'] = vacc_data[vacc_data['State'] == is_available[st]]['State_Code'].iloc[0]
    else:
        sm_st, _ = get_most_similar_word(vacc_data, st.title(), 'State')
        sdf.loc[sdf['Name'] == st, 'State_Code'] = vacc_data[vacc_data['State'] == sm_st]['State_Code'].iloc[0]
sdf.drop(columns=['Name', 'Level'], inplace=True)
sdf = sdf.groupby('State_Code').sum()
sdf

,TOT_P
State_Code,
AN,380581
AP,84580777
AR,1383727
AS,31205576
BR,104099452
CH,1055450
CT,25545198
DL,16787941
DN,586956


In [45]:
state_wise_pop_data = sdf.copy()
state_wise_pop_data = state_wise_pop_data.reset_index()
state_wise_pop_data

,State_Code,TOT_P
0,AN,380581
1,AP,84580777
2,AR,1383727
3,AS,31205576
4,BR,104099452
5,CH,1055450
6,CT,25545198
7,DL,16787941
8,DN,586956
9,GA,1458545


In [46]:
state_wise = pd.merge(vacc_data, state_wise_pop_data, how ='inner', on =['State_Code'])
state_wise.fillna(0, inplace=True)
state_wise

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9,TOT_P
0,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,0,0,0,0,0,0,0,0,0,380581
1,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,0,0,0,0,0,0,0,0,0,380581
2,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,0,0,0,0,0,0,0,0,0,380581
3,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,0,0,0,0,0,0,0,0,0,84580777
4,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,0,0,0,0,0,0,0,0,0,84580777
5,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,0,0,0,0,0,0,0,0,0,84580777
6,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,0,0,0,0,0,0,0,0,0,84580777
7,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,0,0,0,0,0,0,0,0,0,84580777
8,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,0,0,0,0,0,0,0,0,0,84580777
9,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,0,0,0,0,0,0,0,0,0,84580777


In [47]:
ddf = state_wise.copy()
start_date = pd.to_datetime('16/01/2021', format='%d/%m/%Y').date()
end_date = pd.to_datetime('14/08/2021', format='%d/%m/%Y').date()
vacc_col = []
get_original_date=[]
while start_date<=end_date:
    start_date = start_date.strftime('%d/%m/%Y')
    dose1 = str(start_date)+'.3'
    dose2 = str(start_date)+'.4'
    
    #Convert back to datetime format
    start_date = start_date.split('.')[0]
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y').date()
    
    if dose1 not in ddf.columns or dose2 not in ddf.columns:
        print("Date not available: ", start_date)
        continue
    ddf[dose1] = ddf[dose1].astype(int)
    ddf[dose2] = ddf[dose2].astype(int)
    get_original_date.append(start_date.strftime('%d/%m/%Y'))
    vacc_col.append(start_date.strftime('%d/%m/%Y')+'_1')
    vacc_col.append(start_date.strftime('%d/%m/%Y')+'_2')
    ddf[start_date.strftime('%d/%m/%Y')+'_1'] = ddf[dose1] 
    ddf[start_date.strftime('%d/%m/%Y')+'_2'] = ddf[dose2] 
    start_date += timedelta(days=1)

In [48]:
ddf

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,10/08/2021_1,10/08/2021_2,11/08/2021_1,11/08/2021_2,12/08/2021_1,12/08/2021_2,13/08/2021_1,13/08/2021_2,14/08/2021_1,14/08/2021_2
0,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,21936,8012,21944,8026,21997,8046,22032,8079,22081,8110
1,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,63415,28677,63422,28719,64826,28854,66474,29021,68043,29281
2,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,131587,58360,133950,58749,136931,59084,139810,59407,142585,59896
3,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,1292062,563906,1298043,569328,1299214,571164,1357367,594250,1364374,597346
4,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,1463591,640538,1464471,640668,1464566,640753,1544219,640773,1565591,640864
5,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,1879353,544608,1881360,545978,1881778,546816,1924155,551251,1927490,552473
6,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,1692429,564197,1705191,569098,1709624,571534,1747460,587644,1758535,592441
7,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,1707961,542621,1713851,545372,1721065,547797,1737195,552836,1756261,557589
8,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,1182041,476527,1187902,478899,1189983,479770,1220728,487236,1242692,492628
9,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,1194897,497477,1207963,504569,1211101,507244,1220659,510952,1231912,515791


In [49]:
dkey_skey_col = ['State_Code', 'TOT_P']
final_col = dkey_skey_col + vacc_col

final_df = ddf[final_col]
final_df.sort_values('State_Code', inplace=True)
final_df

,State_Code,TOT_P,16/01/2021_1,16/01/2021_2,17/01/2021_1,17/01/2021_2,18/01/2021_1,18/01/2021_2,19/01/2021_1,19/01/2021_2,...,10/08/2021_1,10/08/2021_2,11/08/2021_1,11/08/2021_2,12/08/2021_1,12/08/2021_2,13/08/2021_1,13/08/2021_2,14/08/2021_1,14/08/2021_2
0,AN,380581,0,0,0,0,0,0,1,0,...,21936,8012,21944,8026,21997,8046,22032,8079,22081,8110
1,AN,380581,0,0,0,0,0,0,0,0,...,63415,28677,63422,28719,64826,28854,66474,29021,68043,29281
2,AN,380581,23,0,23,0,42,0,88,0,...,131587,58360,133950,58749,136931,59084,139810,59407,142585,59896
15,AP,84580777,195,0,474,0,641,0,1028,0,...,1031381,389947,1034745,391138,1036297,392136,1040376,393774,1043434,395280
13,AP,84580777,259,0,491,0,554,0,1232,0,...,884720,215858,890084,216413,891923,216682,913664,218694,921086,220017
12,AP,84580777,527,0,944,0,1328,0,2019,0,...,1758550,590250,1784398,602216,1787851,603149,1851746,616498,1875574,623159
11,AP,84580777,165,0,404,0,529,0,1061,0,...,1018985,330517,1023127,333494,1024263,334773,1026409,336118,1028569,336972
10,AP,84580777,277,0,727,0,998,0,1907,0,...,1122921,517716,1125571,518370,1142085,523810,1146908,525613,1158076,528689
9,AP,84580777,202,0,868,0,1295,0,1948,0,...,1194897,497477,1207963,504569,1211101,507244,1220659,510952,1231912,515791
14,AP,84580777,207,0,436,0,426,0,1329,0,...,1572889,422306,1577655,422972,1614683,427762,1622149,428813,1642547,431963


In [50]:
def get_statewise_df(state_final_df, st):
    ddf = state_final_df.copy()
    ddf = ddf[ddf['State_Code'] == st]
    ddf = ddf.groupby('State_Code').sum()
    

    d1 = [c for c in ddf.columns[1::2]]
    d2 = [c for c in ddf.columns[2::2]]
    dose1_df = ddf.loc[:, d1]
    dose2_df = ddf.loc[:, d2]
    
    #Vaccination Rate as per last week data
    vacc_rate_per_week = dose1_df.iloc[0][-8:][-1] - dose1_df.iloc[0][-8:][0]

    return dose1_df.iloc[0][-1], dose2_df.iloc[0][-1], vacc_rate_per_week


In [51]:
state_final_df = final_df.copy()
for st in state_wise_pop_data['State_Code'].unique():
    if st not in state_final_df['State_Code'].unique(): continue
    d1, d2, vacc_rate_per_week = get_statewise_df(state_final_df, st)
    state_wise_pop_data.loc[state_wise_pop_data['State_Code'] == st, 'dose1'] = d1
    state_wise_pop_data.loc[state_wise_pop_data['State_Code'] == st, 'dose2'] = d2
    state_wise_pop_data.loc[state_wise_pop_data['State_Code'] == st, 'rateofvaccination'] = vacc_rate_per_week
state_wise_pop_data = state_wise_pop_data.dropna()
state_wise_pop_data

,State_Code,TOT_P,dose1,dose2,rateofvaccination
0,AN,380581,232709.0,97287.0,17688.0
1,AP,84580777,18516141.0,6485212.0,1039018.0
2,AR,1383727,700977.0,199737.0,12954.0
3,AS,31205576,11430053.0,2442900.0,1240836.0
4,BR,104099452,25167378.0,4859267.0,2221859.0
5,CH,1055450,733579.0,244351.0,46970.0
6,CT,25545198,9383511.0,2775881.0,260467.0
7,DL,16787941,8251002.0,3265001.0,506741.0
8,DN,586956,598380.0,90421.0,18093.0
9,GA,1458545,1112815.0,329909.0,25430.0


In [52]:
st_df = state_wise_pop_data.copy()
st_df = st_df[['State_Code', 'dose1', 'dose2','rateofvaccination', 'TOT_P']]
st_df

,State_Code,dose1,dose2,rateofvaccination,TOT_P
0,AN,232709.0,97287.0,17688.0,380581
1,AP,18516141.0,6485212.0,1039018.0,84580777
2,AR,700977.0,199737.0,12954.0,1383727
3,AS,11430053.0,2442900.0,1240836.0,31205576
4,BR,25167378.0,4859267.0,2221859.0,104099452
5,CH,733579.0,244351.0,46970.0,1055450
6,CT,9383511.0,2775881.0,260467.0,25545198
7,DL,8251002.0,3265001.0,506741.0,16787941
8,DN,598380.0,90421.0,18093.0,586956
9,GA,1112815.0,329909.0,25430.0,1458545


In [53]:
st_df['populationleft'] = st_df['TOT_P'] - st_df['dose1']
st_df['remainingweeks'] = st_df['populationleft']/st_df['rateofvaccination']
st_df['date'] = st_df['remainingweeks'].apply(lambda x: pd.to_datetime('14/08/2021')+timedelta(days=np.round(x)*7) if x>0 else pd.to_datetime('14/08/2021'))
st_df = st_df[['State_Code', 'populationleft', 'rateofvaccination', 'date']]
st_df.rename(columns={'State_Code':'stateid'}, inplace=True)
st_df.set_index('stateid', inplace=True)
state_ratio = st_df.copy()
state_ratio

,populationleft,rateofvaccination,date
stateid,,,
AN,147872.0,17688.0,2021-10-09
AP,66064636.0,1039018.0,2022-11-05
AR,682750.0,12954.0,2022-08-20
AS,19775523.0,1240836.0,2021-12-04
BR,78932074.0,2221859.0,2022-04-23
CH,321871.0,46970.0,2021-10-02
CT,16161687.0,260467.0,2022-10-22
DL,8536939.0,506741.0,2021-12-11
DN,-11424.0,18093.0,2021-08-14


In [54]:
state_ratio.to_csv('../output/complete-vaccination.csv')